In [32]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [33]:
df = pd.read_csv('Updated_CRU.csv')
df.head()

,time,lat,lon,timeseries-tas-monthly-mean,year,month,dayofyear,quarter
0,1901-01-16,26.25,-122.75,16.322001,1901,1,16,1
1,1901-01-16,26.25,-122.25,16.322001,1901,1,16,1
2,1901-01-16,26.25,-121.75,16.322001,1901,1,16,1
3,1901-01-16,26.25,-121.25,16.322001,1901,1,16,1
4,1901-01-16,26.25,-120.75,16.322001,1901,1,16,1


In [34]:
df.shape

(6644352, 8)

In [35]:
df_copy = df.copy()

In [36]:
df_copy['time'] = pd.to_datetime(df_copy['time'])

In [37]:
df_copy.set_index('time', inplace=True)

In [38]:
df_copy = df_copy['timeseries-tas-monthly-mean'].resample('M').mean()

In [39]:
df_copy = df_copy.reset_index()

In [40]:
df_copy['year'] = df_copy['time'].dt.year
df_copy['month'] = df_copy['time'].dt.month
df_copy['quarter'] = df_copy['time'].dt.quarter
df_copy['season'] = df_copy['month'] %12 // 3 + 1

In [41]:
X = df_copy.drop(['timeseries-tas-monthly-mean', 'time'], axis=1)
y = df_copy['timeseries-tas-monthly-mean']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [48]:
X_train.shape, y_train.shape

((1161, 4), (1161,))

In [50]:
feature_scaler = StandardScaler().fit(X_train)
target_scaler = StandardScaler().fit(y_train.values.reshape(-1, 1))

X_train_scaled = feature_scaler.transform(X_train)
y_train_scaled = target_scaler.transform(y_train.values.reshape(-1, 1))

X_test_scaled = feature_scaler.transform(X_test)
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))

In [55]:
class TimeSeriesDataset:
    def __init__(self,X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.__len__()
    
    def __getitem__(self, idx):
        return np.array(self.X[idx], dtype=float), np.array(self.y[idx], dtype=float)